# Leaf Classfication 
A For CZ4041 Machine Learning Assignment from PT3 in AY2018/2019 Semester 2.
The group members are:
- LIU Yiqing
- LUO Bingyi
- TENG He Xu
- WANG Jia
- YI Zhiyue
- ZHAO Ziru

## Import necessary libraries and Define Constants

In [3]:
import cv2
import os
import csv
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

IMAGE_PATH = 'data/images/'
LABEL_PATH = 'data/'
TRAIN_FILE_NAME = 'train.csv'
TEST_FILE_NAME = 'test.csv'
COMMON_HEIGHT_WIDTH = 50


## Load Images and Labels

Load images from the given path

In [4]:
def load_images(path):
    images = os.listdir(path)
    loaded_images = []
    num = len(images)
    
    for i in range(num):
        loaded_images.append(path + images[i])
    return loaded_images

Resize the images to a uniformed size

In [5]:
def image_resize(file_name, hw):
    image = cv2.imread(file_name, 0)
    new_img = cv2.resize(image, (int(hw), int(hw)))
    return new_img

Load labels from CSV

In [6]:
def load_labels(file_path):
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        # Skip the first line
        next(csv_reader)
        # Remove empty lines
        lines = list(line for line in csv_reader if line)
        for line in lines:
            label = {}
            label['id'] = int(line[0])
            label['species'] = line[1]
            labels.append(label)
    
    return labels

In [7]:
def combine_images_and_labels(images, labels):
    id = []
    x = []
    y = []
    for image in images:
        for label in labels:
            if image['id'] == label['id']:
                id.append(image['id'])
                x.append(image['image'])
                y.append(label['species'])
                
    return {
        'id': id,
        'x': x,
        'y': y
    }

## CNN Model Construction

In [8]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, COMMON_HEIGHT_WIDTH, COMMON_HEIGHT_WIDTH, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

### Execution Scripts

In [9]:
images = load_images(IMAGE_PATH)
train_labels = load_labels(LABEL_PATH + TRAIN_FILE_NAME)
resized_images = []

for i in range(len(images)):
    resized_image = image_resize(images[i], COMMON_HEIGHT_WIDTH)
    record = {}
    record['id'] = int(images[i].split('.')[0].split('/')[2])
    record['image'] = resized_image
    resized_images.append(record)
    
## for img in resized_images:
##     plt.imshow(img)
##     plt.show()

train_data = combine_images_and_labels(resized_images, train_labels)
train_x = np.array(train_data['x'])
train_y = np.array(train_data['y'])

In [10]:
leaf_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, 
    model_dir="/tmp/leaf_convnet_model"
)

# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/leaf_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12c715c18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": train_x},
  y=train_y,
  batch_size=100,
  num_epochs=None,
  shuffle=True)

leaf_classifier.train(
  input_fn=train_input_fn,
  steps=20000,
  hooks=[logging_hook])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use keras.layers.conv2d instead.


TypeError: Value passed to parameter 'input' has DataType uint8 not in list of allowed values: float16, bfloat16, float32, float64